# 👨🏽‍🏫 DeciLM-7B-Instruct



In [ ]:
%%capture
!pip install huggingface_hub
!pip install transformers
!pip install accelerate
!pip install bitsandbytes
!pip install ninja
# !pip install flash-attn

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

We load the model with 4-bit quantization in this case since colab usually offers limited T4 GPUs with 16GB VRAM.

In general, we recommend loading the model using `torch_dtype=torch.bfloat16` for best results.

In [ ]:
model_id = 'Deci/DeciLM-7B-instruct'

using_colab_T4_GPU = True
if using_colab_T4_GPU:
  bnb_config = BitsAndBytesConfig(
      load_in_4bit = True,
      bnb_4bit_compute_dtype=torch.bfloat16
  )
  dtype_kwargs = {"quantization_config": bnb_config}
else:
  dtype_kwargs = {"torch_dtype": torch.bfloat16}


model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True,
    **dtype_kwargs
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token = tokenizer.eos_token

In [ ]:
SYSTEM_PROMPT_TEMPLATE ="""
### System:
You are an AI assistant that follows instruction extremely well. Help as much as you can.
### User:
{instruction}
### Assistant:
"""

# Function to construct the prompt using the new system prompt template
def get_prompt_with_template(message: str) -> str:
    return SYSTEM_PROMPT_TEMPLATE.format(instruction=message)

# Function to handle the generation of the model's response using the constructed prompt
def generate_model_response(message: str) -> str:
    prompt = get_prompt_with_template(message)
    inputs = tokenizer(prompt, return_tensors='pt')
    if torch.cuda.is_available():  # Ensure input tensors are on the GPU if model is on GPU
        inputs = inputs.to('cuda')
    output = model.generate(**inputs,
                            max_new_tokens=3000,
                            num_beams=5,
                            no_repeat_ngram_size=4,
                            early_stopping=True
                            )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Function to extract the content after "### Response:"
def extract_response_content(full_response: str) -> str:
    response_start_index = full_response.find("### Assistant:")
    if response_start_index != -1:
        return full_response[response_start_index + len("### Assistant:"):].strip()
    else:
        return full_response

# Main function to get the model's response and extract the content after "### Response:"
def get_response_with_template(message: str) -> str:
    full_response = generate_model_response(message)
    return extract_response_content(full_response)

In [ ]:
# Sample usage
user_message = "How can I teach my toddler son to be more patient and not throw tantrums?"
response = get_response_with_template(user_message)
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Here are some tips to help your toddler learn patience and avoid tantrums:

1. Establish a routine: Toddlers thrive on structure and predictability. Establishing a consistent routine can help them feel more secure and in control, reducing the likelihood of tantrums.

2. Use positive reinforcement: Praise and reward your child when they exhibit patience and self-control. This will encourage them to continue exhibiting these behaviors.

3. Teach coping strategies: Teach your child coping strategies such as deep breathing, counting to 10, or taking a break to calm down.

4. Be patient and consistent: Model patience and consistency in your interactions with your child. This will help them learn these behaviors from you.

5. Limit screen time: Too much screen time can lead to overstimulation and impatience. Limit your child's screen time and encourage them to engage in other activities.

6. Address the root cause: If your child is throwing tantrums in response to a specific situation, try t

In [ ]:
user_message = "How do I write a compelling social media post?"
response = get_response_with_template(user_message)
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Here's a step-by-step guide to help you write a compelling and engaging social media post:

1. Define your goal: Determine what you want to achieve with your social media post. Do you want to inform, educate, entertain, or engage your audience?

2. Identify your audience: Understand your target audience's interests, preferences, and pain points. This will help you tailor your content to resonate with them.

3. Choose the right platform: Select the social media platform that best aligns with your goal and audience. For example, if you're targeting a younger demographic, platforms like Instagram or TikTok may be more suitable.

4. Craft a compelling headline: Your headline is the first thing your audience will see, so make it attention-grabbing and relevant to your content.

5. Use visuals: Incorporate eye-catching visuals like images, videos, or GIFs to make your post more engaging.

6. Keep it concise: Social media users have short attention spans, so keep your post concise and to the 

In [ ]:
user_message = "I have Bisquick flour, eggs, milk, butter. How can I make peanut butter pancakes?"
response = get_response_with_template(user_message)
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Ingredients:
- Bisquick flour
- Eggs
- Milk
- Butter
- Peanut butter

Instructions:
1. Preheat a non-stick pan or griddle over medium heat.
2. In a mixing bowl, whisk together 1 cup of Bisquick, 1 egg, and 1/4 cup of milk.
3. Add 1 tablespoon of butter to the pan and let it melt.
4. Once the butter has melted, pour the batter into the pan.
5. Let the pancake cook for 1-2 minutes or until bubbles form on the surface.
6. Flip the pancakes and cook for an additional 1 minute or until the other side is golden brown.
7. Repeat steps 5 and 6 for the remaining batter.
8. Once all pancakes are cooked, spread a thin layer of peanut better on top of each pancake.
9. Serve and enjoy!


In [ ]:
user_message = "How do I make french toast? Think through it step by step"
response = get_response_with_template(user_message)
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


To make French toast, follow these step-by-step instructions:

1. Preheat the oven to 350 degrees Fahrenheit (180 degrees Celsius).

2. In a mixing bowl, whisk together 4 large eggs, 1 cup of milk, 1 teaspoon of vanilla extract, and a pinch of salt.

3. In a separate bowl, mix together 1/2 cup of granulated sugar and 1 tablespoon of ground cinnamon.

4. In a shallow dish, combine the cinnamon-sugar mixture with 1/4 cup of unsalted butter, melted.

5. Dip each slice of bread into the egg mixture, coating both sides.

6. Place the coated bread slices on a baking sheet lined with parchment paper.

7. Bake the French toast in the preheated oven for 10-15 minutes, or until the bread is golden brown and crispy.

8. Serve the French toast with your favorite toppings, such as maple syrup, powdered sugar, or fresh fruit.

Enjoy your delicious homemade French toast!


In [ ]:
user_message = "Help me come up with an idea for my first machine learning project and tell me the steps to complete it"
response = get_response_with_template(user_message)
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


One possible machine learning project idea is to create a sentiment analysis model that can classify text as positive, negative, or neutral. Here are the steps you can follow to complete this project:

1. **Data Collection**: Gather a dataset of text data, such as movie reviews, product reviews, or social media posts. Make sure the dataset is labeled with the sentiment (positive, negative, neutral) for each text.

2. **Data Preprocessing**: Clean and preprocess the text data by removing stop words, punctuation, and converting all text to lowercase.

3. **Feature Extraction**: Extract features from the preprocessed text, such as bag-of-words, term frequency-inverse document frequency (TF-IDF), or word embeddings.

4. **Model Selection**: Choose a suitable machine learning model for sentiment analysis, such as Naive Bayes, Support Vector Machines (SVM), or Recurrent Neural Networks (RNN).

5. **Training and Evaluation**: Train the model using the labeled dataset and evaluate its performa

In [ ]:
user_message = "Teach me how to make a good trap beat"
response = get_response_with_template(user_message)
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


To make a trap beat, follow these steps:

1. Choose a tempo: Decide on the speed at which you want your beat to play. A common tempo for trap beats is around 100-120 beats per minute (BPM).

2. Select a drum machine or software: You can use a drum machine like the Roland TR-808 or TR-909, or a digital audio workstation (DAW) like Ableton Live, FL Studio, or Logic Pro.

3. Choose a kick drum sound: The kick drum is the most important element of a trap beat. Experiment with different kick drum sounds until you find one that suits your style.

4. Add a snare drum: The snare drum should complement the kick drum. Try different snare drum sounds to find one that works well with your kick drum.

5. Incorporate hi-hats: Hi-hats are an essential part of trap beats. Choose a hi-hat sound that complements the kick and snare drums.

6. Add a clap: A clap sound can add a unique element to your trap beat. Try different clap sounds and see which one fits best.

7. Add a shaker: A shaker can add a sub